<a href="https://colab.research.google.com/github/sowmya-subu/slay-my-text/blob/main/slaymytext_backend.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import os
from getpass import getpass
from datetime import datetime # Import datetime here

GITHUB_USER = "sowmya-subu"     # Replace with your GitHub username
REPO_NAME = "slay-my-text"          # Replace with your repo name
BRANCH = "main"                   # Or "master" if needed
TOKEN = None                      # Stored in memory only during runtime

def setup_repo():
    """Clone repo into Colab if not already cloned."""
    global TOKEN
    if TOKEN is None:
        try:
            from google.colab import userdata
            if TOKEN is None:
                 TOKEN = getpass("🔑 Enter your GitHub Personal Access Token: ")
            else:
                TOKEN = userdata.get('GITHUB_TOKEN') # Attempt to get token from secrets
        except ImportError:
             TOKEN = getpass("🔑 Enter your GitHub Personal Access Token: ")


    if not os.path.exists(REPO_NAME):
        print("📂 Cloning repository...")
        !git clone https://github.com/{GITHUB_USER}/{REPO_NAME}.git
    else:
        print("✅ Repo already exists in this session.")

    # Ensure nbstripout is installed
    try:
        import nbstripout
    except ImportError:
        print("Installing nbstripout...")
        !pip install nbstripout --quiet


def push_to_github(message="Update from Colab"):
    """Clean notebook metadata, commit, and push to GitHub."""
    global TOKEN
    if TOKEN is None:
        try:
            from google.colab import userdata
            TOKEN = userdata.get('GITHUB_TOKEN') # Attempt to get token from secrets
            if TOKEN is None:
                 TOKEN = getpass("🔑 Enter your GitHub Personal Access Token: ")
        except ImportError:
             TOKEN = getpass("🔑 Enter your GitHub Personal Access Token: ")

    original_dir = os.getcwd() # Store the original directory
    repo_path = os.path.join(original_dir, REPO_NAME)

    if not os.path.exists(repo_path):
        print(f"Error: Repository directory '{repo_path}' not found.")
        return # Exit if repo directory doesn't exist

    os.chdir(repo_path) # Change to the repository directory

    try:
        # Remove unnecessary notebook metadata
        print("🧼 Cleaning notebook metadata...")
        !nbstripout *.ipynb

        # Git user configuration (safe email format)
        !git config --global user.email "{GITHUB_USER}@users.noreply.github.com"
        !git config --global user.name "{GITHUB_USER}"

        # Stage all files, including new ones
        print("📦 Staging changes...")
        !git add -A

        # Commit changes, but only if there is something to commit
        commit_check = !git status --porcelain
        if commit_check: # If there's output, there are changes
            print("💾 Committing changes...")
            !git commit -m "{message}"
            print("⬆️ Pushing to GitHub...")
            # Using a temporary file for token to avoid exposing it in process list
            token_file = ".github_token"
            with open(token_file, "w") as f:
                f.write(TOKEN)
            !git push https://{GITHUB_USER}:$(cat {token_file})@github.com/{GITHUB_USER}/{REPO_NAME}.git {BRANCH}
            !rm {token_file} # Remove the temporary file immediately
            print("✅ Changes pushed to GitHub.")
        else:
            print("⚠️ No changes to commit.")

    except Exception as e:
        print(f"An error occurred during git operations: {e}")
    finally:
        os.chdir(original_dir)  # Always return to the original directory

In [4]:
setup_repo()

🔑 Enter your GitHub Personal Access Token: ··········
📂 Cloning repository...
Cloning into 'slay-my-text'...
remote: Enumerating objects: 11, done.
remote: Counting objects: 100% (11/11), done.
remote: Compressing objects: 100% (10/10), done.
remote: Total 11 (delta 1), reused 0 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (11/11), 18.89 KiB | 3.78 MiB/s, done.
Resolving deltas: 100% (1/1), done.
Installing nbstripout...


In [ ]:
from getpass import getpass
from huggingface_hub import login

hf_token = getpass("Enter your Hugging Face token:")
login(token=hf_token)


In [ ]:
#Step 1: Install dependencies
!pip install -q bitsandbytes transformers accelerate

# Step 2: Import necessary libraries
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

# Step 3: Load model and tokenizer
model_name = "mistralai/Mistral-7B-Instruct-v0.2"

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map="auto",              # Automatically use GPU if available
    load_in_4bit=True,              # Efficient 4-bit quantization for free Colab GPU
    torch_dtype=torch.float16
)


In [ ]:
# Step 4: Define simple generation function
def generate_genz_reword(text):
    prompt = f"<|system|>\nYou are a playful, witty Gen Z content creator. Rewrite any input text using Gen Z slang, emojis, and humor while keeping the original meaning.\n\n<|user|>\nRewrite the following text in a Gen Z tone: \"" + text + "\"\n\n<|assistant|>"

    inputs = tokenizer(prompt, return_tensors="pt").to("cuda")
    output = model.generate(**inputs, max_new_tokens=100, do_sample=True, temperature=0.7, top_p=0.9)
    response = tokenizer.decode(output[0], skip_special_tokens=True)

    # Extract only the assistant's response part
    genz_text = response.split('<|assistant|>')[-1].strip()

    # Define basic unsafe words (expand as needed)
    unsafe_words = ["kill", "hate", "suicide", "violence", "racist", "sex", "murder", "abuse"]

    # Check for unsafe content
    if any(word.lower() in genz_text.lower() for word in unsafe_words):
        #log_unsafe(prompt, genz_text)
        return "⚠️ Sorry, I can't reword this text due to unsafe content."

    return genz_text



In [ ]:
# Step 5: Test it! Intermediate test
sample_input = "I'm looking forward to our meeting tomorrow. Let me know if you need anything beforehand."
print("🎤 Original:", sample_input)
print("🎧 Gen Z Style:", generate_genz_reword(sample_input))


In [ ]:
#Create Gradio webapp and launch demo live
!pip install gradio --quiet
import gradio as gr
from datetime import datetime
def capture_feedback(original, reworded, feedback, comment):
    timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    with open("feedback.txt", "a") as f:
        f.write(f"Timestamp: {timestamp}\n")
        f.write(f"Original: {original}\n")
        f.write(f"Reworded: {reworded}\n")
        f.write(f"Feedback: {feedback}\n")
    return "✅ Feedback saved!"

custom_theme = gr.themes.Soft(
    primary_hue="indigo",
    secondary_hue="pink",
    font=[gr.themes.GoogleFont("Fugaz One"), "cursive"],
    radius_size="lg", # Changed from "xl" to "lg"
    spacing_size="lg"
)

demo = gr.Interface(
    fn=generate_genz_reword,
    inputs="text",
    outputs="text",
    title="SlayMyText - Gen Z Rewording",
    description="Rewrite boring text into slay Gen Z speak",
)

with gr.Blocks(theme=custom_theme) as demo:
    gr.Markdown("## 🎤 SlayMyText: Translate Anything to Gen Z 🔥")

    with gr.Row():
        input_text = gr.Textbox(lines=3, label="Enter Your Text")
        output_text = gr.Textbox(lines=3, label="Fire Text")

    run_btn = gr.Button("✨ Slay it, Don't Say it!")

    run_btn.click(fn=generate_genz_reword, inputs=input_text, outputs=output_text)

    gr.Markdown("### 🙏 Did we slay it or nah?")

    with gr.Row():
        feedback = gr.Radio(choices=["🔥 Slayed", "Mid"], label="Your Reaction")
        comment = gr.Textbox(lines=2, label="Optional Comments (e.g., why it's mid)")
        submit_btn = gr.Button("Submit Feedback")
        status = gr.Textbox(label="", interactive=False)

    submit_btn.click(fn=capture_feedback,
                 inputs=[input_text, output_text, feedback, comment],
                 outputs=status)

demo.launch()

In [ ]:
#additional functions to be developed for limiting rate of requests
import time

last_request_time = 0

def rate_limited_slay(text):
    global last_request_time
    if time.time() - last_request_time < 10:  # 10-second delay
        return "Please wait a few seconds between rewordings."
    last_request_time = time.time()
    return slay_my_text(text)


In [ ]:
UNSAFE_KEYWORDS = ["kill", "suicide", "hate", "racist"]

def safe_response(text):
    for word in UNSAFE_KEYWORDS:
        if word in text.lower():
            return "⚠️ Content was flagged as unsafe. Please try a different input."
    return text


In [ ]:
def slay_safe(text):
    response = rate_limited_slay(text)
    return safe_response(response)


In [6]:
#Push this notebook to GitHub
push_to_github("💬 Updated slay-my-text colab notebook "+ datetime.now().strftime("%Y-%m-%d %H:%M:%S") )

#why is this not working



/content/slay-my-text
Error: Repository directory '/content/slay-my-text/slay-my-text' not found.


In [ ]:
!ls -d */



In [ ]:
%cd slay-my-text/

!git fetch origin

!git reset --hard origin/main